In [1]:
import os
import pandas as pd
import plotly.express as px
import folium
from haversine import haversine

In [2]:
home_path = os.path.dirname(os.path.abspath('.'))
df = pd.read_csv(os.path.join(home_path, 'datasets', 'train.csv'))

In [3]:
df.dtypes

ID                              object
Delivery_person_ID              object
Delivery_person_Age             object
Delivery_person_Ratings         object
Restaurant_latitude            float64
Restaurant_longitude           float64
Delivery_location_latitude     float64
Delivery_location_longitude    float64
Order_Date                      object
Time_Orderd                     object
Time_Order_picked               object
Weatherconditions               object
Road_traffic_density            object
Vehicle_condition                int64
Type_of_order                   object
Type_of_vehicle                 object
multiple_deliveries             object
Festival                        object
City                            object
Time_taken(min)                 object
dtype: object

In [4]:
df1 = df.copy()

# 1. convertendo a coluna Age de texto para numero
linhas_selecionadas = df1['Delivery_person_Age'] != 'NaN '
df1 = df1.loc[linhas_selecionadas, :].copy()

df1['Delivery_person_Age'] = df1['Delivery_person_Age'].astype(int)

# 2. convertendo a coluna Ratings de texto para numero decimal (float)
df1['Delivery_person_Ratings'] = df1['Delivery_person_Ratings'].astype(float)

# 3. convertendo a coluna order_date de texto para data
df1['Order_Date'] = pd.to_datetime(df1['Order_Date'], format='%d-%m-%Y')

# 4. convertendo multiple_deliveries de texto para numero inteiro (int)
linhas_selecionadas = df1['multiple_deliveries'] != 'NaN '
df1 = df1.loc[linhas_selecionadas, :].copy()

df1['multiple_deliveries'] = df1['multiple_deliveries'].astype(int)


# 5. Removendo os espacos dentro de strings/texto/object
df1.loc[:, 'ID'] = df1.loc[:, 'ID'].str.strip()
df1.loc[:, 'Road_traffic_density'] = df1.loc[:, 'Road_traffic_density'].str.strip()
df1.loc[:, 'Type_of_order'] = df1.loc[:, 'Type_of_order'].str.strip()
df1.loc[:, 'Type_of_vehicle'] = df1.loc[:, 'Type_of_vehicle'].str.strip()
df1.loc[:, 'City'] = df1.loc[:, 'City'].str.strip()
df1.loc[:, 'Festival'] = df1.loc[:, 'Festival'].str.strip()

# 6. Removend do NaN
df1 = df1.loc[df1['Road_traffic_density'] != 'NaN', :].copy()
df1 = df1.loc[df1['City'] != 'NaN', :].copy()
df1 = df1.loc[df1['Festival'] != 'NaN ', :].copy()

# 7. Limpando a coluna de time taken
df1['Time_taken(min)'] = df1['Time_taken(min)'].apply(lambda x: x.split('(min) ')[1])
df1['Time_taken(min)'] = df1['Time_taken(min)'].astype(int)


In [5]:
df1.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:30:00,11:45:00,conditions Sunny,High,2,Snack,motorcycle,0,No,Urban,24
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,19:45:00,19:50:00,conditions Stormy,Jam,2,Snack,scooter,1,No,Metropolitian,33
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:30:00,08:45:00,conditions Sandstorms,Low,0,Drinks,motorcycle,1,No,Urban,26
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,18:00:00,18:10:00,conditions Sunny,Medium,0,Buffet,motorcycle,1,No,Metropolitian,21
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,13:30:00,13:45:00,conditions Cloudy,High,1,Snack,scooter,1,No,Metropolitian,30


# 1.0 Visão: Empresa

## 1. Quantidade de pedidos por dia

In [6]:
# coluna
cols = ['ID', 'Order_Date']

# seleção de linhas
df_aux = df1.loc[:, cols].groupby('Order_Date').count().reset_index()

# desenhar o grafico de linhas
px.bar(df_aux, x='Order_Date', y='ID')

## 2. Quantidade de pedidos por semana.

In [7]:
# criar a coluna de semana
df1['week_of_year'] = df1['Order_Date'].dt.strftime('%U')
df_aux = df1.loc[:, ['ID', 'week_of_year']].groupby('week_of_year').count().reset_index()

px.line(df_aux, x='week_of_year', y='ID')

## 3. Distribuição dos pedidos por tipo de tráfego.

In [8]:
df_aux = df1.loc[:, ['ID', 'Road_traffic_density']].groupby('Road_traffic_density').count().reset_index()

df_aux['entregas_perc'] = df_aux['ID'] / df_aux['ID'].sum()

px.pie(df_aux, values='entregas_perc', names='Road_traffic_density')

## 4. Comparação do volume de pedidos por cidade e tipo de trafego.

In [9]:
df_aux = df1.loc[:, ['ID', 'City', 'Road_traffic_density']].groupby(['City', 'Road_traffic_density']).count().reset_index()
px.scatter(df_aux, x='City', y='Road_traffic_density', size='ID', color='City')

## 5. A quantidade pedidos por entregador por semana.

In [10]:
# Quantidade de pedidos por semana / Número únicos de entregadores por semana
df_aux01 = df1.loc[:, ['ID', 'week_of_year']].groupby('week_of_year').count().reset_index()
df_aux02 = df1.loc[:, ['Delivery_person_ID', 'week_of_year']].groupby('week_of_year').nunique().reset_index()

df_aux = pd.merge(df_aux01, df_aux02, how='inner')
df_aux['order_by_deliver'] = df_aux['ID'] / df_aux['Delivery_person_ID']

px.line(df_aux, x='week_of_year', y='order_by_deliver')


## 6. A localização central de cada cidade por tipo de tráfego.

In [11]:
df_aux = df1.loc[:, ['City', 'Road_traffic_density', 'Delivery_location_latitude', 'Delivery_location_longitude']].groupby(['City', 'Road_traffic_density']).median().reset_index()

map = folium.Map()

for index, location_info in df_aux.iterrows():
    folium.Marker([location_info['Delivery_location_latitude'],
                   location_info['Delivery_location_longitude']]).add_to(map)

map

# 2.0 Visão: Entregadores

## 1. A menor e maior idade dos entregadores.


In [12]:
# A maior idade dos entregadores
print( "A maior idade é: {}".format(df1.loc[:, 'Delivery_person_Age'].max()))
print( "A menor idade é: {}".format(df1.loc[:, 'Delivery_person_Age'].min()))

A maior idade é: 39
A menor idade é: 20


## 2. A pior e a melhor condição de veículos.


In [13]:
print( "A melhor condição de veículo é: {}".format(df1.loc[:, 'Vehicle_condition'].max()))
print( "A pior condição de veículo é: {}".format(df1.loc[:, 'Vehicle_condition'].min()))

A melhor condição de veículo é: 2
A pior condição de veículo é: 0


## 3. A avaliação média por entregador.


In [14]:
df_avg_ratings_per_delivery = df.loc[:, ['Delivery_person_ID', 'Delivery_person_Ratings']].groupby('Delivery_person_ID').median().reset_index()
df_avg_ratings_per_delivery

,Delivery_person_ID,Delivery_person_Ratings
0,AGRRES010DEL01,4.80
1,AGRRES010DEL02,4.85
2,AGRRES010DEL03,4.70
3,AGRRES01DEL01,4.70
4,AGRRES01DEL02,4.70
...,...,...
1315,VADRES19DEL02,4.70
1316,VADRES19DEL03,4.70
1317,VADRES20DEL01,4.70
1318,VADRES20DEL02,4.70


## 4. A avaliação média e o desvio padrão por tipo de tráfego.


In [15]:
df_avg_std_ranting_by_traffic = (df1.loc[:, ['Delivery_person_Ratings','Road_traffic_density']].groupby('Road_traffic_density')
                                .agg({'Delivery_person_Ratings':['mean', 'std']}))

df_avg_std_ranting_by_traffic.columns = ['delivery_mean', 'delivery_std']
df_avg_std_ranting_by_traffic = df_avg_std_ranting_by_traffic.reset_index()
df_avg_std_ranting_by_traffic

,Road_traffic_density,delivery_mean,delivery_std
0,High,4.652538,0.272699
1,Jam,4.594428,0.329531
2,Low,4.645947,0.337211
3,Medium,4.660335,0.273956


## 5. A avaliação média e o desvio padrão por condições climáticas.


In [16]:
df_avg_std_ranting_by_weather = (df1.loc[:, ['Delivery_person_Ratings','Weatherconditions']].groupby('Weatherconditions')
                                .agg({'Delivery_person_Ratings':['mean', 'std']}))

df_avg_std_ranting_by_weather.columns = ['delivery_mean', 'delivery_std']
df_avg_std_ranting_by_weather = df_avg_std_ranting_by_weather.reset_index()
df_avg_std_ranting_by_weather


,Weatherconditions,delivery_mean,delivery_std
0,conditions Cloudy,4.651945,0.281016
1,conditions Fog,4.653087,0.274974
2,conditions Sandstorms,4.612144,0.310527
3,conditions Stormy,4.612569,0.312621
4,conditions Sunny,4.656164,0.395057
5,conditions Windy,4.616844,0.304087


## 6. Os 10 entregadores mais rápidos por cidade.

In [17]:
df2 = df1.loc[:, ['Delivery_person_ID', 'City','Time_taken(min)']].groupby(['City', 'Delivery_person_ID']).min().sort_values(['City','Time_taken(min)'], ascending=True).reset_index()

df_aux1 = df2.loc[df2['City'] == 'Metropolitian', :].head(10)
df_aux2 = df2.loc[df2['City'] == 'Urban', :].head(10)
df_aux3 = df2.loc[df2['City'] == 'Semi-Urban', :].head(10)

df3 = pd.concat([df_aux1, df_aux2, df_aux3]).reset_index(drop=True)
df3

,City,Delivery_person_ID,Time_taken(min)
0,Metropolitian,AGRRES010DEL03,10
1,Metropolitian,AGRRES07DEL03,10
2,Metropolitian,AGRRES12DEL01,10
3,Metropolitian,AGRRES14DEL01,10
4,Metropolitian,AGRRES17DEL03,10
5,Metropolitian,ALHRES08DEL03,10
6,Metropolitian,ALHRES12DEL01,10
7,Metropolitian,ALHRES13DEL01,10
8,Metropolitian,ALHRES14DEL02,10
9,Metropolitian,ALHRES15DEL02,10


## 7. Os 10 entregadores mais lentos por cidade.

In [18]:
df2 = df1.loc[:, ['Delivery_person_ID', 'City','Time_taken(min)']].groupby(['City', 'Delivery_person_ID']).max().sort_values(['City','Time_taken(min)'], ascending=False).reset_index()

df_aux1 = df2.loc[df2['City'] == 'Metropolitian', :].head(10)
df_aux2 = df2.loc[df2['City'] == 'Urban', :].head(10)
df_aux3 = df2.loc[df2['City'] == 'Semi-Urban', :].head(10)

df3 = pd.concat([df_aux1, df_aux2, df_aux3]).reset_index(drop=True)
df3

,City,Delivery_person_ID,Time_taken(min)
0,Metropolitian,ALHRES010DEL01,54
1,Metropolitian,AURGRES13DEL01,54
2,Metropolitian,BANGRES02DEL01,54
3,Metropolitian,BHPRES16DEL02,54
4,Metropolitian,CHENRES02DEL02,54
5,Metropolitian,CHENRES04DEL01,54
6,Metropolitian,CHENRES07DEL03,54
7,Metropolitian,CHENRES08DEL01,54
8,Metropolitian,CHENRES13DEL02,54
9,Metropolitian,COIMBRES010DEL03,54


# 3.0 Visão: Restaurantes

## 1. A quantidade de entregadores únicos.


In [19]:
devery_unique = len(df1.loc[:, 'Delivery_person_ID'].unique())
print("O numero de entregadores unicos é {}".format(devery_unique))

O numero de entregadores unicos é 1320


## 2. A distância média dos resturantes dos locais de entrega.


In [20]:
cols = ['Restaurant_latitude','Restaurant_longitude', 'Delivery_location_latitude', 'Delivery_location_longitude' ]
df1['distance'] = df1.loc[:, cols].apply(lambda x: 
                                        haversine(
                                            (x['Restaurant_latitude'], x['Restaurant_longitude']),
                                            (x['Delivery_location_latitude'], x['Delivery_location_longitude'])
                                        ),axis=1)
avg_distance = df1['distance'].mean()

print('A distancia media das entregas é: {:.2f} KM.'.format(avg_distance))
                                     

A distancia media das entregas é: 27.35 KM.


## 3. O tempo médio e o desvio padrão de entrega por cidade.


In [21]:
cols = ['City','Time_taken(min)']
df_aux = df1.loc[:, cols].groupby('City').agg({'Time_taken(min)':['mean','std']})
df_aux.columns = ['avg_time', 'std_time']

df_aux = df_aux.reset_index()
df_aux

,City,avg_time,std_time
0,Metropolitian,27.375731,9.164721
1,Semi-Urban,49.710526,2.724992
2,Urban,23.096608,8.894365


## 4. O tempo médio e o desvio padrão de entrega por cidade e tipo de pedido.


In [22]:
cols = ['City','Time_taken(min)', 'Type_of_order']
df_aux = df1.loc[:, cols].groupby(['City', 'Type_of_order']).agg({'Time_taken(min)':['mean','std']})
df_aux.columns = ['avg_time', 'std_time']

df_aux = df_aux.reset_index()
df_aux

,City,Type_of_order,avg_time,std_time
0,Metropolitian,Buffet,27.252377,9.180606
1,Metropolitian,Drinks,27.275139,9.071588
2,Metropolitian,Meal,27.554270,9.249682
3,Metropolitian,Snack,27.415381,9.152578
4,Semi-Urban,Buffet,49.707317,2.731702
5,Semi-Urban,Drinks,49.625000,2.459347
6,Semi-Urban,Meal,50.300000,3.041665
7,Semi-Urban,Snack,49.408163,2.707385
8,Urban,Buffet,23.449257,9.087620
9,Urban,Drinks,23.157568,8.978597


## 5. O tempo médio e o desvio padrão de entrega por cidade e tipo de tráfego.


In [23]:
cols = ['City','Time_taken(min)', 'Road_traffic_density']
df_aux = df1.loc[:, cols].groupby(['City', 'Road_traffic_density']).agg({'Time_taken(min)':['mean','std']})
df_aux.columns = ['avg_time', 'std_time']

df_aux = df_aux.reset_index()
df_aux

,City,Road_traffic_density,avg_time,std_time
0,Metropolitian,High,28.103888,7.935947
1,Metropolitian,Jam,31.937908,9.509890
2,Metropolitian,Low,22.193973,6.827415
3,Metropolitian,Medium,27.694076,8.332809
4,Semi-Urban,High,50.125000,2.629956
5,Semi-Urban,Jam,49.841270,2.717095
6,Semi-Urban,Medium,47.400000,2.011080
7,Urban,High,24.206081,8.535574
8,Urban,Jam,27.905200,10.125553
9,Urban,Low,19.336444,6.351539


## 6. O tempo médio de entrega durantes os Festivais.

In [24]:
cols = ['Time_taken(min)', 'Festival']
df_aux = df1.loc[:, cols].groupby('Festival').agg({'Time_taken(min)':['mean','std']})
df_aux.columns = ['avg_time', 'std_time']

df_aux = df_aux.reset_index()

linhas_selecionadas = df_aux['Festival'] == 'Yes'
df_aux = df_aux.loc[linhas_selecionadas, :]
df_aux

,Festival,avg_time,std_time
2,Yes,45.518607,4.005399
